In [15]:
import json

with open('data/valid_vietnam_tourism.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data = data['data'][0]
data['paragraphs']

[{'context': 'Việt Nam, một viên ngọc quý của Đông Nam Á, nổi bật với vẻ đẹp tự nhiên hùng vĩ và di sản văn hóa phong phú. Từ những bãi biển tuyệt đẹp như Nha Trang và Đà Nẵng cho đến các khu rừng nguyên sinh ở Tây Nguyên, cảnh quan Việt Nam mang đến nhiều trải nghiệm đa dạng cho du khách. Hà Nội, thủ đô ngàn năm văn hiến, hấp dẫn với nét cổ kính và ẩm thực đường phố phong phú, trong khi TP. Hồ Chí Minh lại sôi động và hiện đại, nơi lưu giữ dấu ấn lịch sử và sự phát triển năng động.\n\nViệt Nam còn nổi tiếng với các di sản văn hóa thế giới như Vịnh Hạ Long, phố cổ Hội An và khu di tích Mỹ Sơn, mỗi địa điểm đều có câu chuyện riêng và giá trị lịch sử sâu sắc. Du khách có thể khám phá các lễ hội truyền thống, tìm hiểu về phong tục tập quán của các dân tộc thiểu số, hoặc tham gia vào các lớp học nấu ăn để trải nghiệm ẩm thực phong phú của đất nước.\n\nVới những người yêu thích phiêu lưu, Việt Nam cung cấp nhiều hoạt động thú vị như trekking, lặn biển và khám phá các hang động kỳ vĩ. Bên cạ

In [18]:
para = data['paragraphs']
extracted_para = []
for i in range(len(para)):
    extracted_para.append(para[i]['context'])
joined_para = '\n\n'.join(extracted_para)

with open('data/vietnam_tourism.txt', 'w', encoding='utf-8') as f:
    f.write(joined_para)

In [1]:
with open('tuyen quang.txt', 'r', encoding='utf-8') as f:
    data = f.read()
data

'Khám phá các địa điểm lịch sử nổi bật\nSở hữu tới 546 di tích, trong đó có 435 di tích lịch sử, Tuyên Quang được ví như một “bảo tàng cách mạng sống” – nơi lưu giữ những dấu ấn hào hùng của dân tộc Việt Nam qua từng giai đoạn lịch sử.\n\nKhu di tích lịch sử Tân Trào – Thủ đô khu giải phóng\nTọa lạc tại huyện Sơn Dương, khu di tích Tân Trào là quần thể di tích lịch sử cấp quốc gia đặc biệt, nơi Trung ương Đảng và Chủ tịch Hồ Chí Minh từng sống và làm việc, chuẩn bị cho Cách mạng Tháng Tám năm 1945. Đây là địa điểm không thể bỏ qua khi đến Tuyên Quang.\nCác điểm tham quan tiêu biểu trong khu di tích:\n- Đình Tân Trào: Đình Tân Trào là nơi diễn ra Quốc dân Đại hội ngày 16/8/1945, thông qua 10 chính sách lớn của Việt Minh và quyết định Tổng khởi nghĩa. Kiến trúc truyền thống kết hợp không gian yên bình khiến nơi đây trở thành điểm đến mang đậm tính biểu tượng.\n- Lán Nà Lừa: Nằm sâu trong rừng, Lán Nà Lừa là nơi Bác Hồ sống và làm việc từ cuối tháng 5 đến tháng 8 năm 1945. Công trình bằng

In [2]:
from chromadb import PersistentClient
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import os

# Initialize the embedding model

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# Initialize ChromaDB client
db_dir = 'db/tuyen_quang'
os.makedirs(db_dir, exist_ok=True)
chroma_client = PersistentClient(path=db_dir)

# # The text to process

# text = data

# # 1. Split text into words

# words = text.split()

# # 2. Create chunks of ~350 words

# chunk_size = 350
# chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
chunks = [chunk.strip() for chunk in data.split("\n\n\n") if chunk.strip()]

# 3. Embed each chunk using SentenceTransformer

embeddings = model.encode(chunks, convert_to_numpy=True)

# 4. Create or get a Chroma collection

collection_name = "tuyen_quang"
if collection_name in chroma_client.list_collections():
    collection = chroma_client.get_collection(collection_name)
else:
    collection = chroma_client.create_collection(name=collection_name, embedding_function=None)

# 5. Add chunks and embeddings to Chroma

collection.add(
documents=chunks,
embeddings=embeddings.tolist(),
ids=[f"chunk_{i}" for i in range(len(chunks))]
)

print(f"Added {len(chunks)} chunks to ChromaDB collection '{collection_name}'")



c:\Users\OS TDTS\anaconda3\envs\nlp_rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Added 5 chunks to ChromaDB collection 'tuyen_quang'


In [7]:
# Đọc database từ đầu nè
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.PersistentClient(path=db_dir)
collection = client.get_collection(name='tuyen_quang')

# collection = hello['collection']
query = 'Tuyên Quang du lịch lịch sử có điểm nào hay?'

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
query_embed = model.encode(query, normalize_embeddings=True)

results=collection.query(
    query_embeddings=[query_embed],
    n_results=3,
    include=['metadatas', 'documents', 'distances']
)
print(results['ids'][0])
results['documents'][0]

['chunk_1', 'chunk_4', 'chunk_3']


['Du lịch văn hóa và tâm linh\nTuyên Quang không chỉ là vùng đất giàu truyền thống cách mạng mà còn sở hữu đời sống văn hóa – tâm linh phong phú, gắn liền với tín ngưỡng dân gian, phong tục tập quán và các lễ hội đặc sắc của cộng đồng các dân tộc thiểu số. Đây là một trong những điểm mạnh giúp Tuyên Quang ngày càng thu hút du khách yêu thích trải nghiệm bản địa và tìm hiểu chiều sâu văn hóa vùng cao.\n\nCác đền thờ linh thiêng tại Tuyên Quang\nĐền Hạ (đền Tam Kỳ), Đền Thượng, Đền Ỷ La: Ba ngôi đền nổi tiếng nằm tại thành phố Tuyên Quang, thường được ghép thành tuyến hành hương “Tam đền” thu hút đông đảo du khách mỗi dịp lễ Tết. Các đền này thờ Mẫu Thượng Ngàn, Mẫu Thoải và các vị thần bản địa, gắn liền với tín ngưỡng thờ Mẫu của người Việt. Không gian tĩnh lặng, linh thiêng kết hợp với kiến trúc cổ kính tạo nên điểm đến vừa mang tính tâm linh, vừa có giá trị văn hóa.\nĐền Pác Tạ và đền Pác Vãng (Na Hang): Nằm gần hồ Na Hang, hai ngôi đền này không chỉ là nơi sinh hoạt tín ngưỡng của ng